# Módulo 2 – Limpieza y Transformación de Datos

En esta sesión aprenderás a preparar tus datos antes de un análisis profundo. Trabajaremos con los datasets **`salarios.csv`** y **`paises.csv`** ubicados en `data/raw/`.


##  Objetivos de la sesión
- Detectar y manejar **valores nulos** y **duplicados**.
- Limpiar columnas monetarias y convertir tipos de datos.
- **Unir** (merge) datasets por claves comunes.
- Crear **nuevas columnas** útiles para el análisis.
- Exportar los datos limpios a `data/processed/`.


## Recuerden la Estructura de un proyecto

Una estructura recomendada puede ser:
```
project-name/
├── assest/           # Imagenes, graficos (generalmente no se sube a github)
├── data/             # Datos crudos y procesados
    ├──── raw/   
    ├──── processed/   --> Crear esta carpeta si no la tienen
├── notebooks/        # Notebooks del proyecto
├── utils/            # Funciones auxiliares
├── docs/             # Documentación
├── README.md         # Descripción general
├── environment.yml   # Dependencias usando conda
└── requirements.txt  # Dependencias usando pip

---
## 1 · Importar librerías y cargar datasets
Comenzamos cargando Pandas y leyendo los archivos CSV desde la carpeta `data/raw/`.


### 📦 Librerías que usaremos
Importamos módulos clave para:

* **pandas / numpy** → manipulación numérica y tabular  
* **matplotlib / seaborn** → gráficos rápidos (los usaremos más adelante)  
* **pathlib** → manejar rutas de forma independiente del sistema operativo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

### 🔍 Ubicar el archivo de salarios
Utilizamos la constante `DATA_RAW_DIR` (definida en `utils.paths`) para construir
la ruta absoluta a **salarios.csv**.  
Esto evita escribir paths “hard-codeados” y hace el notebook portable.

In [ ]:
#Importar la carpeta de datos raw desde paths
from cursos.analisis_datos.utils.paths import DATA_RAW_DIR

#definir un path o ruta con Path
path_salarios = DATA_RAW_DIR / "salarios.csv"
path_salarios = str(path_salarios)

print(f"Ruta del archivo salarios: {path_salarios}")
print(f"existe archivo salarios: {Path(path_salarios).exists()}")

### 📥 Carga del dataset de salarios
Leemos el archivo con `pd.read_csv()`.  
* `sep=','` porque es un CSV estándar.  
* `encoding='utf-8'` asegura que caracteres latinos/acentos se lean bien.

In [ ]:
df_salarios = pd.read_csv(filepath_or_buffer=path_salarios, sep=',', encoding='utf-8')

In [ ]:
df_salarios.head(7)

---
## 2 · Explorar la estructura del dataset
### 2.1 Información general
Revisamos tipos de datos, cantidad de filas/columnas y valores nulos.


### 🏷️ Inspección rápida del DataFrame
`df.info()` resume:
* Columnas y tipos (`object`, `int64`, etc.)  
* Filas no-nulas por columna → detectamos valores faltantes (`pais`, `empresa`)  
* Memoria ocupada

In [ ]:
df_salarios.info()

### 2.2 Estadísticas descriptivas rápidas


### 📊 Estadísticas descriptivas
`df.describe()` genera medidas (count, mean, std…) solo para columnas
numéricas. Aquí vemos *edad* y *años en la empresa*.  
Esto ayuda a detectar outliers y entender la dispersión.

In [ ]:
df_salarios.describe()

---
## 3 · Limpiar columnas monetarias
Las columnas `sueldo_anual` y `bono_anual` tienen símbolos de moneda. Eliminamos `$` y `€` y convertimos a `float`.


In [ ]:
df_salarios.head()

Revisar las columnas `sueldo_anual` y `bono_anual` pues tienen los simbolos de moneda

### 💲 Columnas monetarias originales
Previsualizamos `sueldo_anual` y `bono_anual` antes de limpiarlas.
Observa que contienen símbolos (`$`, `€`) 

In [ ]:
# escribe tu codigo aqui
df_salarios[['sueldo_anual', 'bono_anual']]

Renombrar las columnas usando `.rename`

```python
columns={
        "sueldo_anual": "sueldo_anual_dolares",
        "bono_anual": "bono_anual_euros"
    }
```

In [ ]:
# escribe tu codigo aqui


Revisar si las columnas cambiaron de nombre con `.columns`

In [ ]:
# escribe tu codigo aqui


Reemplazar el simbolo de dólar `$` por  `''` utilizando el método `.replace()` 

### ✂️ Eliminar símbolo de dólar
`str.replace('$','')` quita el carácter pero **deja las comas**.
Aún es tipo `object` (string); falta:
1. eliminar comas,
2. convertir a `float`.

In [ ]:
# escribe tu codigo aqui


Cambiar el tipo de dato de sueldo_anual_dolares con `.astype(float)`

> **Nota:** se produce un `ValueError` porque todavía hay **celdas vacías**
  (`''`) o valores con comas que impiden la conversión directa a `float`.

In [ ]:

df_salarios['sueldo_anual_dolares'] = df_salarios['sueldo_anual_dolares'].astype(float)

### Funcionó? o Cuál es el error?

##### Tu respuesta aqui

reemplazar las celdas vacias `''`

### 🛠️ Reemplazar strings vacíos por `NaN`
Usamos `.replace('', np.nan)` para que pandas los reconozca como faltantes
y permita la conversión numérica en el siguiente paso.

In [ ]:
# escribe tu codigo aqui


Cambiar el tipo de datos con `.astype(float)`

### 🔄 Convertir a numérico
Ahora que no hay símbolos ni strings vacíos, `astype(float)` funciona.
El resultado es una columna `float64` lista para cálculos.

In [ ]:
# escribe tu codigo aqui


Revisar que valores de `sueldo_anual_dolares` tienen el valor `np.nan`

### 📌 Ver cuántos salarios quedaron como `NaN`
Esto nos indica cuántos registros tendrán que imputarse
o eliminarse en el siguiente bloque de limpieza.

In [ ]:
# escribe tu codigo aqui


### 🔍 Inspeccionar filas con salario faltante
Listamos las 7 filas donde `sueldo_anual_dolares` es `NaN`.
Podremos decidir si:
* imputar con la mediana por cargo/país,  
* o eliminar si los valores son críticos.

In [ ]:
df_salarios[df_salarios['sueldo_anual_dolares'].isna()]

Ahora el mismo proceso con el símbolo de euros `€`

In [ ]:
df_salarios['bono_anual_euros'] = (df_salarios['bono_anual_euros'].str.replace('€', ''))

In [ ]:
df_salarios['bono_anual_euros'] = (df_salarios['bono_anual_euros'].astype(float))

In [ ]:
df_salarios[df_salarios['bono_anual_euros'].isna()]

In [ ]:
df_salarios[['sueldo_anual_dolares', 'bono_anual_euros']].head()

---
## 4 · Detección y manejo de valores nulos
### 4.1 Contar nulos por columna


In [ ]:
df_salarios.isnull().sum()

> **Estrategias comunes:**
- **Eliminar** filas/columnas con muchos nulos (`dropna`).
- **Imputar** con media/mediana/moda (`fillna`).
- **Revisar** si los nulos tienen significado en tu análisis.


Revisemos el caso de la columna `['pais']`

In [ ]:
# escribe tu codigo aqui

In [ ]:
df_salarios = df_salarios.dropna(subset=["pais"])

In [ ]:
df_salarios.isnull().sum()

Ahora revisemos el caso de la columna `["nombre_de_la_empresa"]`

In [ ]:
df_salarios[df_salarios["nombre_de_la_empresa"].isna()]

No es necesario eliminar esos datos, pues podemos clasificarlos como `"Empresa Desconocida"`

In [ ]:
# escribe tu codigo aqui

In [ ]:
df_salarios[df_salarios["nombre_de_la_empresa"].isna()]

In [ ]:
df_salarios.isnull().sum()

Revisemos el caso de `["sueldo_anual_dolares"]`

In [ ]:
df_salarios[df_salarios["sueldo_anual_dolares"].isna()]

Al no existir el registro de sueldos, la decision que tomamos fue eliminar esas filas con `dropna()`

In [ ]:
df_salarios = df_salarios.dropna(subset=["sueldo_anual_dolares"])

In [ ]:
df_salarios.isnull().sum()

Ahora no tenemos valores nulos

---
## 5 · Detección y manejo de duplicados
Buscamos registros exactos repetidos.


In [ ]:
duplicados = # escribe tu codigo aqui
print(f'Duplicados encontrados: {duplicados.sum()}')

In [ ]:
df_salarios = df_salarios.drop_duplicates()

In [ ]:
duplicados = df_salarios.duplicated()
print(f'Duplicados encontrados: {duplicados.sum()}')

In [ ]:
df_salarios.info()

### 🔢 Reiniciar índices
`reset_index(drop=True)` crea un nuevo índice consecutivo (0 → n-1) y descarta el índice anterior.  
Útil después de eliminar filas o reordenar el DataFrame para evitar huecos o duplicados en la numeración.

In [ ]:
df_salarios = df_salarios.reset_index(drop=True)

In [ ]:
df_salarios.info()

### 💱 Conversión de Euros a Dólares  
Definimos una **tasa de cambio fija** `TASA_EUR_USD = 1.10` (1 € → 1.10 US$)  
y creamos la columna `bono_anual_dolares` multiplicando cada valor en  
`bono_anual_euros` por esa tasa. Esta operación es vectorizada: se aplica  
a toda la columna sin necesidad de bucles y deja ambos montos disponibles  
para comparaciones o visualizaciones futuras.

In [ ]:
df_salarios.head()

In [ ]:
# 1 € ≈ 1.10 USD
TASA_EUR_USD = 1.10

In [ ]:
df_salarios["bono_anual_dolares"] = (
    df_salarios["bono_anual_euros"] * TASA_EUR_USD
)

In [ ]:
df_salarios[["bono_anual_euros", "bono_anual_dolares"]]

### 📏 Ajuste de precisión  
Redondeamos `bono_anual_dolares` a **una cifra decimal** (`round(1)`) para  
homogeneizar el formato de los valores monetarios y evitar “ruido” de  
centavos insignificantes al presentar o agrupar los datos.

In [ ]:
# escribe tu codigo aqui

In [ ]:
df_salarios[["bono_anual_euros", "bono_anual_dolares"]]

In [ ]:
df_salarios.info()

## 6 · Cargar el dataframe de países


In [ ]:
#definir un path o ruta con Path
path_paises = DATA_RAW_DIR / "paises.csv"
path_paises = str(path_paises)

print(f"Ruta del archivo paises: {path_paises}")
print(f"existe archivo paises: {Path(path_paises).exists()}")

In [ ]:
df_paises = pd.read_csv(filepath_or_buffer=path_paises, sep=',', encoding='utf-8')

In [ ]:
df_paises.head()

In [ ]:
df_paises['pais'].unique()

In [ ]:
df_paises['pais'].nunique()

In [ ]:
df_salarios['pais'].unique()

In [ ]:
df_salarios['pais'].nunique()

In [ ]:
paises_en_df_paises = set(df_paises["pais"])
paises_en_df_paises

In [ ]:
paises_en_df_salarios = set(df_salarios["pais"])
paises_en_df_salarios

### 🔍 Identificar discrepancias entre DataFrames

1. **`paises_faltantes_en_df_paises`**  
   Conjunto de países que **están en `df_paises` pero no aparecen** en `df_salarios`.

2. **`paises_faltantes_en_df_salarios`**  
   Conjunto inverso: países presentes en `df_salarios`, ausentes en `df_paises`.

3. **`paises_faltantes`**  
   Unión (`|`) de ambos conjuntos → lista global de **todos los países que no coinciden** entre los dos DataFrames.  
   Útil para decidir si imputar, descartar o investigar la fuente de datos.

In [ ]:
paises_faltantes_en_df_paises = paises_en_df_paises - paises_en_df_salarios
paises_faltantes_en_df_paises

In [ ]:
paises_faltantes_en_df_salarios =  paises_en_df_salarios - paises_en_df_paises
paises_faltantes_en_df_salarios

In [ ]:
paises_faltantes = paises_faltantes_en_df_paises | paises_faltantes_en_df_salarios
paises_faltantes

In [ ]:
df_paises[df_paises['pais'].isin(paises_faltantes)]

### 🗑️ Eliminar registros específicos  
Usamos `drop(index=[1, 8, 9])` para remover las filas cuyo índice es 1, 8 y 9  
(Brasil, Ecuador y Bolivia en este caso). Esto excluye esos países del  
DataFrame `df_paises` antes de realizar uniones o análisis posteriores.

In [ ]:
# escribe tu codigo aqui

In [ ]:
df_paises.reset_index(drop=True, inplace=True)

In [ ]:
df_paises

### 💲 Limpieza y conversión de columnas monetarias  
Seleccionamos las columnas `PIB` e `ingreso_per_capita` y:

1. **`replace({'\\$': '', ',': ''}, regex=True)`**  
   - Elimina el símbolo de dólar (`$`) y las comas de miles (`,`).  
   - `regex=True` permite usar patrones de expresión regular.

2. **`astype(float)`**  
   - Convierte las cadenas ya limpias a valores numéricos (`float64`).

Resultado: ambas columnas quedan listas para cálculos y comparaciones
estadísticas sin símbolos ni separadores.

In [ ]:
df_paises[["PIB", "ingreso_per_capita"]] = (df_paises[["PIB", "ingreso_per_capita"]].replace({'\\$': '', ',': ''}, regex=True).astype(float))

In [ ]:
df_paises.head()

In [ ]:
df_paises.isnull().sum()

---
## 7 · Unir datasets (`merge`)
Unimos `df_salarios` con `df_paises` por la columna `pais` para enriquecer la información.


In [ ]:
# escribe tu codigo aqui

In [ ]:
df_salarios_pais.head()

---
## 8 · Crear columnas derivadas


### 💰 Métrica de compensación total  
Creamos `compensacion_total` sumando el salario anual (`sueldo_anual_dolares`)  
y el bono anual convertido a dólares (`bono_anual_dolares`).  
Esta columna refleja cuánto percibe realmente cada empleado en un año,  
facilitando análisis comparativos por país, cargo o empresa.

In [ ]:
# escribe tu codigo aqui

In [ ]:
df_salarios_pais.head()

### 🏆 Clasificación de senioridad  
Creamos la columna `senioridad` con `np.where`:

* **Condición** → `anos_en_la_empresa > 5`  
  - Si el empleado lleva **más de 5 años**, se le etiqueta como **"Senior"**.  
  - En caso contrario, se marca como **"Junior"**.

Así obtenemos una variable categórica que nos ayudará a comparar
compensaciones y otros indicadores entre niveles de experiencia.

In [ ]:
# escribe tu codigo aqui

In [ ]:
df_salarios_pais[['nombre', 'apellido', 'anos_en_la_empresa','compensacion_total', 'senioridad']].head(10)

### 🔎 Revisión de valores nulos por columna  
`df_salarios_pais.isna().sum()` muestra cuántos `NaN` hay en cada columna  
tras el merge. Los resultados indican que los registros de **Costa Rica**  
aún presentan nulos en `capital`, `PIB`, `cantidad_de_habitantes` e  
`ingreso_per_capita`. Conviene imputar o investigar estos campos antes de  
continuar con el análisis para evitar sesgos o errores en cálculos agregados.

In [ ]:
df_salarios_pais.isna().sum()

In [ ]:
df_salarios_pais[df_salarios_pais['pais'] == 'Costa Rica']

### 🔍 Filtrar filas de Costa Rica  
Creamos `mask_cr` para identificar las filas donde `pais == "Costa Rica"` y,  
con `loc`, mostramos únicamente esos registros. Así podemos inspeccionar  
qué columnas siguen con `NaN` y decidir cómo imputarlas o ajustarlas.

In [ ]:
mask_cr = df_salarios_pais["pais"] == "Costa Rica"
df_salarios_pais.loc[mask_cr]

### 📝 Diccionario con datos oficiales de Costa Rica  
Definimos `datos_cr` con los valores verificados para imputar en las filas  
de Costa Rica:

* **capital:** San José  
* **cantidad_de_habitantes:** 5 150 000  
* **PIB:** 86 500 000 000 USD  
* **ingreso_per_capita:** 14 319 USD

Estos números reemplazarán los `NaN` correspondientes en el DataFrame.

In [ ]:
datos_cr = {
    "capital": "San José",
    "cantidad_de_habitantes": 5150000,
    "PIB": 86500000000,
    "ingreso_per_capita": 14319
}

### 🛠️ Imputar valores faltantes para Costa Rica  
Recorremos `datos_cr` y, para cada columna:

1. **Ubicamos** únicamente las filas de Costa Rica (`mask_cr`).  
2. **`fillna(valor)`** reemplaza solo los `NaN` con el dato oficial correspondiente.  
3. Asignamos el resultado de vuelta a la misma columna, dejando intactos los
   registros de otros países.

De este modo, Costa Rica queda completa sin alterar los valores ya correctos.

In [ ]:
# escribe tu codigo aqui

In [ ]:
df_salarios_pais.loc[mask_cr]

### ✅ Verificación final de nulos  
Ejecutamos `df_salarios_pais.isnull().sum()` para confirmar que **todas las  
columnas ahora muestran 0 valores faltantes**. Nuestro DataFrame está listo  
para pasar al Análisis Exploratorio de Datos sin riesgos de errores por `NaN`.

In [ ]:
# escribe tu codigo aqui

In [ ]:
df_salarios_pais.info()

---
## 9 · Exportar dataset limpio


In [ ]:
from cursos.analisis_datos.utils.paths import DATA_PROCESSED_DIR

In [ ]:
salarios_pais_path = DATA_PROCESSED_DIR / "salarios_pais.csv"
salarios_pais_path = str(salarios_pais_path)

print(f"Ruta del archivo salarios_paises: {salarios_pais_path}")
print(f"existe archivo salarios_pais: {Path(salarios_pais_path).exists()}")

### 💾 Exportar dataset limpio  
`to_csv(..., index=False, sep=',', encoding='utf-8')` guarda `df_salarios_pais` como un CSV en la ruta definida por `salarios_pais_path`, sin incluir la columna de índice y usando codificación UTF-8.  
La línea `print()` confirma en pantalla la ubicación del archivo generado, facilitando su localización para futuros análisis o compartición.

In [ ]:
# escribe tu codigo aqui

In [ ]:
print(f' Dataset limpio exportado a {salarios_pais_path}')

In [ ]:
print(f"existe archivo salarios_pais: {Path(salarios_pais_path).exists()}")

---
## 10 · Resumen 
En esta práctica aprendiste a:
- Limpiar columnas con símbolos.
- Detectar/llenar nulos y eliminar duplicados.
- Realizar un **merge** y enriquecer tu información.
- Crear métricas derivadas y exportar datos limpios.

